# Imports

In [3]:
from __future__ import absolute_import, division, print_function
import sys
import os
import glob

from skimage.io import imread
from keras.preprocessing.image import array_to_img, img_to_array, load_img

import numpy as np

# Our Preprocessing module----
sys.path.append('..')
from defectfinder import preprocessing as pre

# Generating all the augmented images along with the noise for all the defect types at once - 

In [24]:
# the path of the folder (Multislice) which includes different type of (theoretical) defects
theo_path = ".././Multislice"
folders=os.listdir(theo_path) 
# folders is the list of subfolders' names (here it is the types of defects' names:Mo,Sw,Vs2,Vw,W2S2,Ws)

for folder in folders:  #traversing all subfolders(types of defects) in MULTISLICE
    if '.' in folder:   #make sure it will not traverse file like '.DS_store'
        continue
        
    path = theo_path +"/"+folder
    
    files = os.listdir(path)
    #files is the list of all raw_images under each type of defect
    for file in files: #traversing all raw_images in the defect folders
        if file.endswith(".png"):
            img = imread(path+'/'+file, as_grey = True) # loading the image
            pre.AugmentImage(img, theo_path, folder+'_Augmented2', folder, 20) # Augmenting using the function above
    
    
    noisy_path = ".././Multislice"+"/"+folder+'/'+folder+'_Augmented2'  
    # the path of data_Augmented folder
    image=os.listdir(noisy_path)
    # image is the list all the augmented data
    for i in image:
        if i.endswith(".png"):
            img = imread(noisy_path+'/'+i, as_grey = True)
            pre.AddNoiseImage(img, folder, noisy_path, i)   #for each augmented image generate 4 images with gaussion,s&p,poisson and speckle noise
            
        else: 
            pass
        
       

C:\Users\Sarthak\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


# For augmenting individual defect types and adding noise separately, use the code below- 

### This is written to be used for adding additional defect types individually...

In [25]:
# the path of the folder (Multislice) which includes different type of (theoretical) defects
theo_path = '.././Multislice'

# Define path to the defect folder
path = '.././' + 'Multislice' + '/'

"""Change defect folder name here"""
folder = "Mo"

"""Change number of augmneted images here"""

N = 250 # Number of Augmented images (Total images after augmentation and adding noise will be ~5*N)

# Augmenting using the function
for filename in glob.iglob(path + folder + '/*.png'):
    
    img = load_img(filename, grayscale = True)
    
    pre.AugmentImage(img, theo_path, folder + '_Augmented2', folder , N)
    

# Adding noise to the augmented images

noisy_path = ".././Multislice"+"/"+folder+'/'+ folder+'_Augmented2'  

# the path of data_Augmented folder
image=os.listdir(noisy_path)

# image is the list all the augmented data
for i in image:
    if i.endswith(".png"):
        
        img = imread(noisy_path+'/'+i, as_grey = True)
        
        pre.AddNoiseImage(img, folder, noisy_path, i)   #for each augmented image generate 4 images with gaussion,s&p,poisson and speckle noise
            
    else: 
        pass

C:\Users\Sarthak\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))


# Generate training and test dataset
## for all the augmented images(with or without noise), save as numpy array
* Only 6 types of defects {'Sw':0,'Mo':1,'W2s2':2,'Vw':3,'Vs2':4,'Ws':5} are considered 
* x is not preprocessed(normalized or centered), y is converted into category

In [4]:
## read in all the augmented and noisy images
path = ".././Multislice"
folders=os.listdir(path) 
data=[]    #all the images will be stored in this list.
defecttype=[]    #all the defect type corresponding to each image store in this list
defects={'Sw':0,'Mo':1,'W2s2':2,'Vw':3,'Vs2':4,'Ws':5}     # each int represents a defect type

for folder in folders:  #traversing all subfolders(types of defects) in MULTISLICE
    if '.' in folder:   #make sure it will not traverse file like '.DS_store'
        continue
    
    noisy_path = ".././Multislice"+"/"+folder+'/'+folder+'_Augmented' 
    filelist = glob.glob(noisy_path+'/*.png')
    data.extend([img_to_array(load_img(fname, grayscale = True)) for fname in filelist])
    defecttype.extend([defects[folder]]*len(filelist))

In [5]:
np.array(data).shape
# This is the total number of data we have

(51125, 64, 64, 1)

In [ ]:
#train test split - 80% train and 20% test set

x_train, x_test, y_train, y_test = train_test_split(np.array(data), defecttype, test_size = 0.2, random_state = 28956)

y_train = keras.utils.to_categorical(y_train, len(defects))
y_test = keras.utils.to_categorical(y_test, len(defects))

# save the np array under .././dataset
if not os.path.exists("../dataset"):
    os.makedirs("../dataset")
    
np.save("../dataset/x_train.npy",x_train)
np.save("../dataset/y_train.npy",y_train)
np.save("../dataset/x_test.npy",x_test)
np.save("../dataset/y_test.npy",y_test)
